Rebuild street graph
====================
Loads a street graph and reallocates roads space, e.g., into a network of one-way streets.

Prerequisite: Preparing the simplified street graph

In [ ]:
import snman
from snman.constants import *

PERIMETER = '_debug'

# Set these paths according to your own setup
data_directory = 'C:/Users/lballo/polybox/Research/SNMan/SNMan Shared/data_v2/'
inputs_path = data_directory + 'inputs/'
process_path = data_directory + 'process/' + PERIMETER + '/'

export_path = data_directory + 'outputs/' + '_debug' + '/'
#export_path = data_directory + 'outputs/' + PERIMETER + '/'

Loading data
------------
Loads the prepared simplified street graph, as well as other datasets such as perimeters, rebuilding regions and measurement regions

In [ ]:
print('Load street graph')
G = snman.io.load_street_graph(process_path + 'street_graph_edges.gpkg', process_path + 'street_graph_nodes.gpkg')

In [ ]:
print('Load perimeters')
perimeters_gdf = snman.io.load_perimeters(inputs_path + 'perimeters/perimeters.shp')

print('Load rebuilding regions')
# Polygons that define which streets will be reorganized
rebuilding_regions_gdf = snman.io.load_rebuilding_regions(
    inputs_path + 'rebuilding_regions/rebuilding_regions.gpkg'
)

print('Load measurement regions')
# Polygons that define areas where network measures will be calculated
measurement_regions_gdf = snman.io.load_measurement_regions(
    inputs_path + 'measurement_regions/measurement_regions.gpkg'
)

Rebuilding regions
------------------
Rebuilds the streets in each region, in an order specified in the rebuilding regions file.

In [ ]:
print('Rebuild regions')
snman.rebuilding.rebuild_regions(
    G,
    rebuilding_regions_gdf,
    verbose=True,
    export_L=(export_path + 'L_edges.gpkg', export_path + 'L_nodes.gpkg'),
    export_H=(export_path + 'H_edges.gpkg', export_path + 'H_nodes.gpkg')
)

Reorder lanes
-------------
Improve the arrangement of lanes after rebuilding. This step also optimizes the design of the cycling infrastructure, e.g., merging parallel cycling lanes into one or separating cyclists and pedestrians from mixed paths if possible.

In [ ]:
print('Reorder lanes')
snman.space_allocation.reorder_lanes(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)

Export
------
Saves the resulting datasets to the disk. Use the *snman_detailed.qgz* file to view them in QGIS.

In [ ]:
if 1:
    print('Export network without lanes')
    snman.io.export_street_graph(G, export_path + 'street_graph_edges.gpkg', export_path + 'street_graph_nodes.gpkg')

if 1:
    print('Export network with lanes')
    SCALING = 3
    snman.io.export_street_graph_with_lanes(G, KEY_LANES_DESCRIPTION, export_path + 'lane_geometries.shp', scaling=SCALING)
    snman.io.export_street_graph_with_lanes(G, KEY_LANES_DESCRIPTION_AFTER, export_path + 'lane_geometries_after.shp', scaling=SCALING)

if 0:
    print('Export OSM XML')
    snman.io.export_osm_xml(G, export_path + 'new_network.osm', EXPORT_OSM_TAGS, uv_tags=True)